# GAM Coach: Binary Classification

In this notebook, we will learn how to generate *diverse* and *customizable* counterfactual explanations for Generalized Additive Models (GAMs).

## 2. German


In [2]:
%load_ext autoreload
%autoreload 2

import gamcoach as coach

import numpy as np
import pandas as pd

import dice_ml
import urllib.request
import json
import pickle

from interpret.glassbox import ExplainableBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from collections import Counter
from tqdm import tqdm

SEED = 101221

In [28]:
german_data = pd.read_csv('./data/german_credit.csv')
german_data.head()


# From https://github.com/amirhk/mace/blob/01e6a405ff74e24dc3438a005cd60892154d189d/_data_main/fair_adult_data.py
german_attrs = [
    "account_check_status",
    "duration_in_month",
    "credit_history",
    "purpose",
    "credit_amount",
    "savings",
    "present_emp_since",
    "installment_as_income_perc",
    "personal_status_sex",
    "other_debtors",
    "present_res_since",
    "property",
    "age",
    "other_installment_plans",
    "housing",
    "credits_this_bank",
    "job",
    "people_under_maintenance",
    "telephone",
    "foreign_worker",
]


selected_features = [i for i in range(1, german_data.shape[1])]

x_all = german_data.to_numpy()[:, selected_features]
y_all_ints = german_data.iloc[:, 0].tolist()
y_all = np.array(y_all_ints)

german_feature_names = np.array(german_attrs)
german_cont_indexes = [1, 4, 12]
german_feature_types = [
    "continuous" if i in german_cont_indexes else "categorical"
    for i in range(len(german_feature_names))
]


for c in range(x_all.shape[1]):

    if c in german_cont_indexes:
        x_all[:, c] = x_all[:, c].astype(float)
    else:
        x_all[:, c] = x_all[:, c].astype(str)


x_train, x_test, y_train, y_test = train_test_split(
    x_all, y_all, test_size=0.3, random_state=SEED
)

In [29]:
# Train an EBM classifier
german_ebm = ExplainableBoostingClassifier(feature_names=german_feature_names, feature_types=german_feature_types)
german_ebm.fit(x_train, y_train)

ExplainableBoostingClassifier(feature_names=['account_check_status',
                                             'duration_in_month',
                                             'credit_history', 'purpose',
                                             'credit_amount', 'savings',
                                             'present_emp_since',
                                             'installment_as_income_perc',
                                             'personal_status_sex',
                                             'other_debtors',
                                             'present_res_since', 'property',
                                             'age', 'other_installment_plans',
                                             'housing', 'credits_this_bank',
                                             'job', 'people_under_maintenan...
                                             'continuous', 'categorical',
                                             'categorical', 

In [30]:
# Evaluate our model

y_pred = german_ebm.predict(x_test)
y_pred_prob = german_ebm.predict_proba(x_test)[:, 1]

print(Counter(y_pred))
print()

accuracy = metrics.accuracy_score(y_test, y_pred)
auc = metrics.roc_auc_score(y_test, y_pred_prob)
f1 = metrics.f1_score(y_test, y_pred)
recall = metrics.recall_score(y_test, y_pred)
precision = metrics.precision_score(y_test, y_pred)
balanced_accuracy = metrics.balanced_accuracy_score(y_test, y_pred)

confusion_matrix = metrics.confusion_matrix(y_test, y_pred)

tn = confusion_matrix[0, 0]
fn = confusion_matrix[1, 0]
fp = confusion_matrix[0, 1]
tp = confusion_matrix[1, 1]
specificity = tn / (tn + fp)

temp = ('accuracy: {:.4f} \nauc: {:.4f} \nrecall:{:.4f} \nprecision: {:.4f} '+
    'specificity: {:.4f} \nf1: {:.4f}\nbalanced accuracy: {:.4f}')
print(temp.format(accuracy, auc, recall, precision, specificity, f1, balanced_accuracy))
print()

print('confusion matrix:\n', confusion_matrix)

Counter({0: 239, 1: 61})

accuracy: 0.7533 
auc: 0.7827 
recall:0.4198 
precision: 0.5574 specificity: 0.8767 
f1: 0.4789
balanced accuracy: 0.6482

confusion matrix:
 [[192  27]
 [ 47  34]]


In [31]:
# Find an interesting data point
# We can focus on test cases where our model rejcets the application (y_hat = 0)
reject_index = y_pred == 0
x_reject = x_test[reject_index, :]
y_pred_reject = y_pred[reject_index]

explain_df = pd.DataFrame(x_reject)
explain_df.columns = german_feature_names

for c in range(len(german_feature_names)):
    name = german_feature_names[c]
    if c in german_cont_indexes:
        explain_df[name] = explain_df[name].astype(float)
    else:
        explain_df[name] = explain_df[name].astype(str)

reject_df = pd.DataFrame(np.hstack((x_reject, y_pred_reject.reshape(-1, 1))))
reject_df.columns = german_feature_names.tolist() + ['prediction']
reject_df.head()

,account_check_status,duration_in_month,credit_history,purpose,credit_amount,savings,present_emp_since,installment_as_income_perc,personal_status_sex,other_debtors,...,property,age,other_installment_plans,housing,credits_this_bank,job,people_under_maintenance,telephone,foreign_worker,prediction
0,>= 200 DM / salary assignments for at least 1 ...,15.0,existing credits paid back duly till now,domestic appliances,2327.0,... < 100 DM,... < 1 year,2,female : divorced/separated/married,none,...,real estate,25.0,none,own,1,unskilled - resident,1,none,yes,0
1,no checking account,9.0,existing credits paid back duly till now,car (new),3577.0,100 <= ... < 500 DM,1 <= ... < 4 years,1,male : single,guarantor,...,real estate,26.0,none,rent,1,skilled employee / official,2,none,no,0
2,no checking account,36.0,critical account/ other credits existing (not ...,car (new),3535.0,... < 100 DM,4 <= ... < 7 years,4,male : single,none,...,"if not A121/A122 : car or other, not in attrib...",37.0,none,own,2,skilled employee / official,1,"yes, registered under the customers name",yes,0
3,no checking account,24.0,critical account/ other credits existing (not ...,car (used),4042.0,unknown/ no savings account,4 <= ... < 7 years,3,male : single,none,...,if not A121 : building society savings agreeme...,43.0,none,own,2,skilled employee / official,1,"yes, registered under the customers name",yes,0
4,no checking account,24.0,critical account/ other credits existing (not ...,car (used),6842.0,unknown/ no savings account,1 <= ... < 4 years,2,male : single,none,...,if not A121 : building society savings agreeme...,46.0,none,own,2,management/ self-employed/ highly qualified em...,2,"yes, registered under the customers name",yes,0


In [32]:
def evaluate_cf(x1, x2, feature_names, cont_mads):
    cur_distance = 0
    feature_weight = 1 / len(feature_names)
    changed_feature = 0

    for i in range(len(feature_names)):
        cur_name = feature_names[i]

        # If the current feature is continuous
        if cur_name in cont_mads:
            if cont_mads[cur_name] > 0:
                cur_distance += (
                    feature_weight * abs(float(x2[i]) - float(x1[i])) / cont_mads[cur_name]
                )
            else:
                cur_distance += (
                    feature_weight * abs(float(x2[i]) - float(x1[i]))
                )
        else:
            cur_distance += 1 if x1[i] != x2[i] else 0

        if x1[i] != x2[i]:
            changed_feature += 1

    return cur_distance, changed_feature


### 1.1. GAM Coach

In [33]:
features_to_vary = german_feature_names.tolist()

In [34]:
german_cont_indexes

[1, 4, 12]

In [35]:
explain_df.iloc[0, :].to_numpy()

array(['>= 200 DM / salary assignments for at least 1 year', 15.0,
       'existing credits paid back duly till now', 'domestic appliances',
       2327.0, '... < 100 DM', '... < 1 year ', '2',
       'female : divorced/separated/married', 'none', '3', 'real estate',
       25.0, 'none', 'own', '1', 'unskilled - resident', '1', 'none',
       'yes'], dtype=object)

In [36]:
my_coach = coach.GAMCoach(german_ebm, x_train, adjust_cat_distance=False)

cfs = []
for i in tqdm(range(explain_df.shape[0])):

    # try:
    cf = my_coach.generate_cfs(
        explain_df.iloc[i, :].to_numpy(),
        total_cfs=1,
        verbose=0,
        categorical_weight=1
        # Some continuous features need to have integer values in practice
        # continuous_integer_features=["open_acc", "total_acc", "mort_acc", "fico_score"],
    )

    cfs.append([explain_df.iloc[i, :], cf.to_df().iloc[0, :]])
    # except:
    #     cfs.append(None)

100%|██████████| 239/239 [03:51<00:00,  1.03it/s]


In [37]:
pickle.dump(cfs, open('./data/german-coach-cfs.pkl', 'wb'))

In [39]:
distances, feature_nums, failed_indexes = [], [], []

for i in range(len(cfs)):
    if cfs[i] is None:
        failed_indexes.append(i)
        continue

    x1 = cfs[i][0]
    x2 = cfs[i][1][:-1]

    cur_distance, changed_feature = evaluate_cf(x1, x2, german_feature_names, my_coach.cont_mads)
    
    distances.append(cur_distance)
    feature_nums.append(changed_feature)

In [40]:
print(
    "coach",
    "\ndistance: ",
    np.mean(distances),
    "\nnum: ",
    np.mean(feature_nums),
    "\nfails: ",
    len(failed_indexes),
)


coach 
distance:  1.1392360658284462 
num:  2.096234309623431 
fails:  0


### 1.2. Genetic Algorithm

In [43]:
dice_df = pd.DataFrame(x_train)
dice_df.columns = german_feature_names
dice_df['status'] = y_train

feature_names_cont = german_feature_names[german_cont_indexes]

for c in feature_names_cont:
    dice_df[c] = dice_df[c].astype(float)

dice_data = dice_ml.Data(
    dataframe=dice_df,
    continuous_features=feature_names_cont.tolist(),
    outcome_name='status'
)

dice_model = dice_ml.Model(model=german_ebm, backend='sklearn')

In [44]:
exp = dice_ml.Dice(dice_data, dice_model, method='genetic')

dice_rejected_df = explain_df.copy()
for c in feature_names_cont:
    dice_rejected_df[c] = dice_rejected_df[c].astype(float)

explanation_generic = exp.generate_counterfactuals(
    dice_rejected_df,
    total_CFs=1,
    features_to_vary=features_to_vary,
    # posthoc_sparsity_param=0.2,
    verbose=False,
    desired_class="opposite")

100%|██████████| 239/239 [01:12<00:00,  3.31it/s]


In [45]:
g_distances, g_feature_nums, g_failed_indexes = [], [], []

for i in range(len(explanation_generic.cf_examples_list)):
    if explanation_generic.cf_examples_list[i].final_cfs_df is None:
        g_failed_indexes.append(i)
        continue

    x1 = dice_rejected_df.iloc[i, :]
    x2 = explanation_generic.cf_examples_list[i].final_cfs_df.iloc[0, :]

    cur_distance, changed_feature = evaluate_cf(x1, x2, german_feature_names, my_coach.cont_mads)
    
    g_distances.append(cur_distance)
    g_feature_nums.append(changed_feature)


In [46]:
print(
    "generic",
    "\ndistance: ",
    np.mean(g_distances),
    "\nnum: ",
    np.mean(g_feature_nums),
    "\nfails: ",
    len(g_failed_indexes),
)


generic 
distance:  6.85726886394003 
num:  9.330543933054393 
fails:  0


## KD Trees

In [48]:
exp = dice_ml.Dice(dice_data, dice_model, method='kdtree')

dice_rejected_df = explain_df.copy()
for c in feature_names_cont:
    dice_rejected_df[c] = dice_rejected_df[c].astype(float)

explanation_tree = exp.generate_counterfactuals(
    dice_rejected_df,
    total_CFs=1,
    features_to_vary=features_to_vary,
    # posthoc_sparsity_param=0.2,
    verbose=False,
    desired_class="opposite")

100%|██████████| 239/239 [10:53<00:00,  2.74s/it]


In [50]:
k_distances, k_feature_nums, k_failed_indexes = [], [], []

for i in range(len(explanation_tree.cf_examples_list)):
    if explanation_tree.cf_examples_list[i].final_cfs_df is None:
        k_failed_indexes.append(i)
        continue

    if explanation_tree.cf_examples_list[i].final_cfs_df.shape == (0, 0):
        k_failed_indexes.append(i)
        continue

    x1 = dice_rejected_df.iloc[i, :]
    x2 = explanation_tree.cf_examples_list[i].final_cfs_df.iloc[0, :]

    cur_distance, changed_feature = evaluate_cf(x1, x2, german_feature_names, my_coach.cont_mads)
    
    k_distances.append(cur_distance)
    k_feature_nums.append(changed_feature)


In [51]:
print(
    "tree",
    "\ndistance: ",
    np.mean(k_distances),
    "\nnum: ",
    np.mean(k_feature_nums),
    "\nfails: ",
    len(k_failed_indexes),
)


tree 
distance:  7.356541890019207 
num:  9.94142259414226 
fails:  0


In [53]:
dice_rejected_df.shape

(239, 20)